In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import os
import sys
from os import listdir
from os.path import isfile, join, exists
from scipy.spatial.distance import pdist, squareform
import scipy.io as sio
import scipy.stats as stats
from tqdm import tqdm
import time
import pymatreader
from IPython.core.debugger import set_trace
import copy
import h5py

import matplotlib.pyplot as plt

In [2]:
homedir = '/media/tarrlab/scenedata2/BOLD5000_GLMs/git/'

In [7]:
# LOAD INFO FOR 1000 SHARED STIMULI in NSD
nsddesign_fn = join(homedir, 'notebooks', 'nsd_stim_info_merged.csv')

df = pd.read_csv(nsddesign_fn)

b5k_df = df.loc[np.logical_and(df['BOLD5000'] == True, df['shared1000'] == True)]
b5k_df



,Unnamed: 0,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
2950,2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.093750,2950,False,True,True,1,...,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566
2990,2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,2990,False,True,True,1,...,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711
3049,3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.000000,3049,False,True,True,1,...,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697
3077,3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,3077,False,True,True,1,...,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537
3146,3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,3146,False,True,True,1,...,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807
3157,3157,584,train2017,"(0, 0, 0.1671875, 0.1671875)",0.142857,3157,False,True,True,1,...,8592,1125,7990,8592,1125,7990,8592,1125,7990,8592
3164,3164,605,train2017,"(0, 0, 0.125, 0.125)",0.000000,3164,False,True,True,1,...,17602,11592,17581,17602,11592,17581,17602,11592,17581,17602
3171,3171,625,train2017,"(0, 0, 0.1515625, 0.1515625)",0.000000,3171,False,True,True,1,...,13388,1798,12368,13388,1798,12368,13388,1798,12368,13388
3181,3181,650,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,3181,False,True,True,1,...,26679,7737,7866,26679,7737,7866,26679,7737,7866,26679
3386,3386,1308,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,3386,False,True,True,1,...,23599,11569,11797,23599,11569,11797,23599,11569,11797,23599


In [20]:
cocoids = b5k_df['cocoId'].values

for subj in ['CSI1','CSI2','CSI3']:

    b5design_fn = join(homedir,'notebooks','sub-' + subj + '_imgnames.txt')
    b5design = np.loadtxt(b5design_fn,dtype='str')
    
    for cocoid in tqdm(cocoids):
        
        found = False
        for img in b5design:
            if str(cocoid) in img:
                found = True
        
        assert(found is True)


100%|██████████| 1000/1000 [00:06<00:00, 144.50it/s]


In [ ]:
# plan:

# for each version/subject in both datasets, we want to make an RDM (1000x1000)
# assuming that all repetitions have been averaged together

# for each dataset, we want an array that's (subjects, repetitions, 1000 images, X, Y, Z)
# this is prob too big to fit in memory

# the approach will need to entail loading one session at a time, loading the stim list
# identifying which indices of that session belong to the shared1000, and then placing the trials into the correct positions

In [ ]:
for subj in ['CSI1','CSI2','CSI3']:

    b5design_fn = join(homedir,'notebooks','sub-' + subj + '_imgnames.txt')
    b5design = np.loadtxt(b5design_fn,dtype='str')
    